# Attempt to do a cluster recomandation model with pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import countDistinct

In [3]:
spark = SparkSession.builder.appName("KNN_Recommender") \
    .config("spark.driver.memory", "100g") \
    .config("spark.executor.memory", "100g") \
    .getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/18 08:53:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# 1. Lire les ratings
ratings = spark.read.csv("../data/raw/ratings.csv", header=True, inferSchema=True)
ratings = ratings.select("userId", "movieId", "rating")

In [6]:
nb_films = ratings.select("movieId").distinct().count()
print(f"Nombre de films avec au moins une note : {nb_films}")

Nombre de films avec au moins une note : 84432


In [7]:
spark.conf.set("spark.sql.pivotMaxValues", 85000)

In [8]:
user_item = ratings.groupBy("userId").pivot("movieId").avg("rating").fillna(0)

In [9]:
from pyspark.sql.functions import count

# Top 100 films les plus populaires
top_movies = (
    ratings.groupBy("movieId")
    .agg(count("userId").alias("num_ratings"))
    .orderBy(col("num_ratings").desc())
    .limit(100)
    .select("movieId")
    .rdd.flatMap(lambda x: x)
    .collect()
)

user_item = ratings.filter(col("movieId").isin(top_movies))
user_item = user_item.groupBy("userId").pivot("movieId").avg("rating").fillna(0)


In [10]:
feature_cols = user_item.columns[1:]  # Sauf userId
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
user_vectors = assembler.transform(user_item).select("userId", "features")


In [14]:
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt

cost = []
K = list(range(100000, 101000))   # Par exemple pour k=2 à 10

for k in K:
    kmeans = KMeans(k=k, seed=42)
    model = kmeans.fit(user_vectors)
    wssse = model.summary.trainingCost
    print(f"k={k}  WSSSE={wssse}")
    cost.append(wssse)

# Tracer la courbe du "coude"
plt.plot(K, cost, marker='o')
plt.xlabel('Nombre de clusters (k)')
plt.ylabel('WSSSE')
plt.title('Elbow Method for Optimal k')
plt.show()


ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/opt/python/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/python/lib/python3.13/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "/opt/python/lib/python3.13/socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
KeyboardInterrupt
ERROR:py4j.clientserver:Exception occurred while shutting down connection
Traceback (most recent call last):
  File "/opt/python/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/python/lib/python3.13/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decod

KeyboardInterrupt: 

# ABORT BECAUSE THERE IS TOO MUCH CLUSTER. TRY KNN INSTEAD